In [1]:
import pandas as pd
from pymongo import MongoClient
import re
import csv


### Connection avec mongodb


In [2]:
client = MongoClient("localhost:27017")
database = client.scrapyPipeline
movies = database.movies
serie = database.serie
# movies.drop()
# serie.drop()

### Nettoyage des données

In [3]:
# df_movie = pd.read_csv("csv/items.csv")
# df_serie = pd.read_csv("csv/serie.csv")

In [4]:
# df_movie["titre_original"].replace(to_replace="Original title: ", value='', regex=True,inplace=True)


In [5]:
# df_movie["duree"].replace(to_replace=",", value='', regex=True,inplace=True)
# df_serie["duree"].replace(to_replace=",", value='', regex=True,inplace=True)
def convertion_duree(s):
    
    split_cell = re.split("\s", s)
    duree = 0
    for elt in split_cell:
        if "h" in elt:
            duree = int(re.split("h", elt)[0]) *60 +duree
        elif "m" in elt:
            duree =int(re.split("m", elt)[0]) +duree
    return duree
    




# df_movie['duree'] = df_movie['duree'].apply(lambda x :  convertion_duree(str(x)))
# df_serie['duree'] = df_serie['duree'].apply(lambda x :  convertion_duree(str(x)))



In [6]:
# df_movie.to_csv("csv/items.csv",index=False)
# df_serie.to_csv("csv/serie.csv",index=False)

### Import des données dans mongodb

In [7]:
# header = ["acteurs","date","descriptions","duree","genre","pays","public","score","titre","titre_original","url"]
# csvfile = open('csv/items.csv', 'r')
# reader = csv.DictReader( csvfile )

# for each in reader:
#     row={}
#     for field in header:

#         if field in  ["acteurs","genre","pays"]:
#             row[field]=each[field].split(",")
#         elif field in ["date","duree"]:
#             row[field]=int(each[field])
#         elif field =="score":
#             row[field]=float(each[field])
#         else :
#             row[field]=each[field]
        
#     movies.insert_one(row)

In [8]:
# header = ["acteurs","date","descriptions","duree","genre","pays","public","score","titre","titre_original"]
# csvfile = open('csv/serie.csv', 'r')
# reader = csv.DictReader( csvfile )

# for each in reader:
#     row={}
#     for field in header:

#         if field in  ["acteurs","genre","pays"]:
#             row[field]=each[field].split(",")
#         elif field in ["duree"]:
#             row[field]=int(each[field])
#         elif field =="score":
#             row[field]=float(each[field])
#         else :
#             row[field]=each[field]  
        
#     serie.insert_one(row)

## question n°1

In [9]:
movies.find().sort("duree",-1).limit(1)[0]

{'_id': ObjectId('639b41ed0913ab014f14c6a5'),
 'titre': 'Gone with the Wind',
 'titre_original': '',
 'score': 8.2,
 'genre': ['Drama', 'Romance', 'War'],
 'date': 1939,
 'duree': 238,
 'descriptions': 'The manipulative daughter of a Georgia plantation owner conducts a turbulent romance with a roguish profiteer during the American Civil War and Reconstruction periods.',
 'acteurs': ['Clark Gable', 'Vivien Leigh', 'Thomas Mitchell'],
 'public': 'Passed',
 'pays': ['United States'],
 'url': 'https://m.media-amazon.com/images/M/MV5BYjUyZWZkM2UtMzYxYy00ZmQ3LWFmZTQtOGE2YjBkNjA3YWZlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_QL75_UX190_CR0,5,190,281_.jpg',
 'langue': ['English']}

## Question n°2

In [10]:
five_movies=movies.find().sort("score",-1).limit(5)
for movie in five_movies:
    print(movie)

{'_id': ObjectId('639b41de0913ab014f14c63c'), 'titre': 'The Shawshank Redemption', 'titre_original': '', 'score': 9.3, 'genre': ['Drama'], 'date': 1994, 'duree': 142, 'descriptions': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'acteurs': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], 'public': 'R', 'pays': ['United States'], 'url': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_QL75_UX190_CR0,0,190,281_.jpg', 'langue': ['English']}
{'_id': ObjectId('639b42010913ab014f14c735'), 'titre': 'The Godfather', 'titre_original': '', 'score': 9.2, 'genre': ['Crime', 'Drama'], 'date': 1972, 'duree': 175, 'descriptions': 'The aging patriarch of an organized crime dynasty in postwar New York City transfers control of his clandestine empire to his reluctant youngest son.', 'acteurs': ['Marlon Brando', 'Al Pacino', 'James Caan'], 'public': 'R', 'pa

## Question n°3


In [11]:
Freeman_movies = movies.find({"acteurs":{"$in":["Morgan Freeman"]}})
for movie in Freeman_movies:
    print(movie)

{'_id': ObjectId('639b41de0913ab014f14c63c'), 'titre': 'The Shawshank Redemption', 'titre_original': '', 'score': 9.3, 'genre': ['Drama'], 'date': 1994, 'duree': 142, 'descriptions': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'acteurs': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], 'public': 'R', 'pays': ['United States'], 'url': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_QL75_UX190_CR0,0,190,281_.jpg', 'langue': ['English']}
{'_id': ObjectId('639b41eb0913ab014f14c698'), 'titre': 'Million Dollar Baby', 'titre_original': '', 'score': 8.1, 'genre': ['Drama', 'Sport'], 'date': 2004, 'duree': 132, 'descriptions': 'A determined woman works with a hardened boxing trainer to become a professional.', 'acteurs': ['Hilary Swank', 'Clint Eastwood', 'Morgan Freeman'], 'public': 'PG-13', 'pays': ['United States'], 'url': 'https://m.media-am

In [12]:
Cruise_movies = movies.find({"acteurs":{"$in":["Tom Cruise"]}})
for movie in Cruise_movies:
    print(movie)

{'_id': ObjectId('639b41f80913ab014f14c6f3'), 'titre': 'Top Gun: Maverick', 'titre_original': '', 'score': 8.4, 'genre': ['Action', 'Drama'], 'date': 2022, 'duree': 130, 'descriptions': "After thirty years, Maverick is still pushing the envelope as a top naval aviator, but must confront ghosts of his past when he leads TOP GUN's elite graduates on a mission that demands the ultimate sacrifice from those chosen to fly it.", 'acteurs': ['Tom Cruise', 'Jennifer Connelly', 'Miles Teller'], 'public': 'PG-13', 'pays': ['United States'], 'url': 'https://m.media-amazon.com/images/M/MV5BZWYzOGEwNTgtNWU3NS00ZTQ0LWJkODUtMmVhMjIwMjA1ZmQwXkEyXkFqcGdeQXVyMjkwOTAyMDU@._V1_QL75_UX190_CR0,0,190,281_.jpg', 'langue': ['English']}


## Question n°4

In [13]:
horror_movies = movies.find({"genre":{"$in":["Horror"]}}).sort("score",-1).limit(3)
for movie in horror_movies:
    print(movie)

{'_id': ObjectId('639b41fd0913ab014f14c713'), 'titre': 'Alien', 'titre_original': '', 'score': 8.5, 'genre': ['Horror', 'Sci-Fi'], 'date': 1979, 'duree': 117, 'descriptions': 'The crew of a commercial spacecraft encounter a deadly lifeform after investigating an unknown transmission.', 'acteurs': ['Sigourney Weaver', 'Tom Skerritt', 'John Hurt'], 'public': 'R', 'pays': ['United Kingdom', 'United States'], 'url': 'https://m.media-amazon.com/images/M/MV5BOGQzZTBjMjQtOTVmMS00NGE5LWEyYmMtOGQ1ZGZjNmRkYjFhXkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_QL75_UX190_CR0,3,190,281_.jpg', 'langue': ['English']}
{'_id': ObjectId('639b41e00913ab014f14c646'), 'titre': 'Psycho', 'titre_original': '', 'score': 8.5, 'genre': ['Horror', 'Mystery', 'Thriller'], 'date': 1960, 'duree': 109, 'descriptions': "A Phoenix secretary embezzles $40,000 from her employer's client, goes on the run and checks into a remote motel run by a young man under the domination of his mother.", 'acteurs': ['Anthony Perkins', 'Janet Leigh', '

In [14]:
drama_movies = movies.find({"genre":{"$in":["Drama"]}}).sort("score",-1).limit(3)
for movie in drama_movies:
    print(movie)

{'_id': ObjectId('639b41de0913ab014f14c63c'), 'titre': 'The Shawshank Redemption', 'titre_original': '', 'score': 9.3, 'genre': ['Drama'], 'date': 1994, 'duree': 142, 'descriptions': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'acteurs': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], 'public': 'R', 'pays': ['United States'], 'url': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_QL75_UX190_CR0,0,190,281_.jpg', 'langue': ['English']}
{'_id': ObjectId('639b42010913ab014f14c735'), 'titre': 'The Godfather', 'titre_original': '', 'score': 9.2, 'genre': ['Crime', 'Drama'], 'date': 1972, 'duree': 175, 'descriptions': 'The aging patriarch of an organized crime dynasty in postwar New York City transfers control of his clandestine empire to his reluctant youngest son.', 'acteurs': ['Marlon Brando', 'Al Pacino', 'James Caan'], 'public': 'R', 'pa

In [15]:
comedy_movies = movies.find({"genre":{"$in":["Comedy"]}}).sort("score",-1).limit(3)
for movie in comedy_movies:
    print(movie)

{'_id': ObjectId('639b41fe0913ab014f14c71e'), 'titre': 'Life Is Beautiful', 'titre_original': 'La vita è bella', 'score': 8.6, 'genre': ['Comedy', 'Drama', 'Romance'], 'date': 1997, 'duree': 116, 'descriptions': 'When an open-minded Jewish waiter and his son become victims of the Holocaust, he uses a perfect mixture of will, humor, and imagination to protect his son from the dangers around their camp.', 'acteurs': ['Roberto Benigni', 'Nicoletta Braschi', 'Giorgio Cantarini'], 'public': 'PG-13', 'pays': ['Italy'], 'url': 'https://m.media-amazon.com/images/M/MV5BYmJmM2Q4NmMtYThmNC00ZjRlLWEyZmItZTIwOTBlZDQ3NTQ1XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_QL75_UX190_CR0,1,190,281_.jpg', 'langue': ['Italian', 'German', 'English']}
{'_id': ObjectId('639b41fd0913ab014f14c716'), 'titre': 'Modern Times', 'titre_original': '', 'score': 8.5, 'genre': ['Comedy', 'Drama', 'Romance'], 'date': 1936, 'duree': 87, 'descriptions': 'The Tramp struggles to live in modern industrial society with the help of a young ho

## Question 6

In [16]:
american_movies = movies.find().sort("score",-1).limit(100)
count=0
for movie in american_movies:
    if "United States" in movie["pays"]:
        count +=1
pourcentage = count/100
print(pourcentage*100)

73.0


In [17]:
french_movies = movies.find({"pays":{"$in":["France"]}}).sort("score",-1).limit(100)
for movie in french_movies:
    print(movie["titre"],movie["score"])

City of God 8.6
Léon: The Professional 8.5
The Pianist 8.5
Cinema Paradiso 8.5
The Intouchables 8.5
Capernaum 8.4
The Boat 8.4
Amadeus 8.4
The Lives of Others 8.4
Incendies 8.3
A Separation 8.3
Amélie 8.3
The Passion of Joan of Arc 8.2
The Wages of Fear 8.2
Casino 8.2
Ran 8.2
The Father 8.2
The 400 Blows 8.1
La haine 8.1
Before Sunset 8.1
Wild Tales 8.1


In [18]:
top_100 = movies.find().sort("score", -1)[:100]
france = 0
us = 0
for i in top_100:
    if "France" in i['pays']  :
        france += 1
    elif "United States" in i['pays'] :
        us += 1
f"Dans le Top 100 des films : US ({us}%) FR ({france}%)"

'Dans le Top 100 des films : US (71%) FR (12%)'

## Question n°7


In [19]:
liste_genre =[]
list_movies = movies.find()
for movie in list_movies:
    genre_movie = movie["genre"]
    for genre in genre_movie:
        if not genre in liste_genre:
            liste_genre.append(genre)
print(liste_genre, len(liste_genre))

['Drama', 'Action', 'Crime', 'Mystery', 'Adventure', 'Thriller', 'Sci-Fi', 'Animation', 'Comedy', 'Horror', 'Family', 'Fantasy', 'Biography', 'History', 'Music', 'Romance', 'Film-Noir', 'War', 'Sport', 'Western', 'Musical'] 21


In [20]:
dict_temps_genre={}
for genre in liste_genre:
    temps_total = 0
    list_movies = movies.find({"genre":{"$in":[genre]}})
    nb_movies=list_movies.count()
    
    for movie in list_movies:
        temps_total = movie["duree"] + temps_total

    dict_temps_genre[genre] =  temps_total/nb_movies
print(dict_temps_genre)


{'Drama': 134.63636363636363, 'Action': 133.92156862745097, 'Crime': 131.7843137254902, 'Mystery': 124.61290322580645, 'Adventure': 129.41666666666666, 'Thriller': 121.03333333333333, 'Sci-Fi': 131.7, 'Animation': 100.95652173913044, 'Comedy': 106.32608695652173, 'Horror': 120.6, 'Family': 114.53846153846153, 'Fantasy': 123.57142857142857, 'Biography': 147.06896551724137, 'History': 156.0, 'Music': 134.25, 'Romance': 118.47826086956522, 'Film-Noir': 110.0, 'War': 140.43478260869566, 'Sport': 122.0, 'Western': 149.33333333333334, 'Musical': 103.0}


/tmp/ipykernel_45063/3980943192.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  nb_movies=list_movies.count()


In [21]:
avg_genre= (movies.aggregate([
    {"$unwind":"$genre"},
        {"$group": {
            "_id": "$genre","duree": { "$avg": "$duree" }
        }
    }
]))
for i in avg_genre:
    print(i)

{'_id': 'Musical', 'duree': 103.0}
{'_id': 'Family', 'duree': 114.53846153846153}
{'_id': 'Animation', 'duree': 100.95652173913044}
{'_id': 'Western', 'duree': 149.33333333333334}
{'_id': 'Adventure', 'duree': 129.41666666666666}
{'_id': 'History', 'duree': 156.0}
{'_id': 'Biography', 'duree': 147.06896551724137}
{'_id': 'Sport', 'duree': 122.0}
{'_id': 'Mystery', 'duree': 124.61290322580645}
{'_id': 'Comedy', 'duree': 106.32608695652173}
{'_id': 'Drama', 'duree': 134.63636363636363}
{'_id': 'Film-Noir', 'duree': 110.0}
{'_id': 'Crime', 'duree': 131.7843137254902}
{'_id': 'Music', 'duree': 134.25}
{'_id': 'Action', 'duree': 133.92156862745097}
{'_id': 'Horror', 'duree': 120.6}
{'_id': 'Romance', 'duree': 118.47826086956522}
{'_id': 'Sci-Fi', 'duree': 131.7}
{'_id': 'War', 'duree': 140.43478260869566}
{'_id': 'Thriller', 'duree': 121.03333333333333}
{'_id': 'Fantasy', 'duree': 123.57142857142857}


In [22]:
for key,value in dict_temps_genre.items():
    print(f"Le temps moyen du genre {key} est {round(value,2)} minutes ")

Le temps moyen du genre Drama est 134.64 minutes 
Le temps moyen du genre Action est 133.92 minutes 
Le temps moyen du genre Crime est 131.78 minutes 
Le temps moyen du genre Mystery est 124.61 minutes 
Le temps moyen du genre Adventure est 129.42 minutes 
Le temps moyen du genre Thriller est 121.03 minutes 
Le temps moyen du genre Sci-Fi est 131.7 minutes 
Le temps moyen du genre Animation est 100.96 minutes 
Le temps moyen du genre Comedy est 106.33 minutes 
Le temps moyen du genre Horror est 120.6 minutes 
Le temps moyen du genre Family est 114.54 minutes 
Le temps moyen du genre Fantasy est 123.57 minutes 
Le temps moyen du genre Biography est 147.07 minutes 
Le temps moyen du genre History est 156.0 minutes 
Le temps moyen du genre Music est 134.25 minutes 
Le temps moyen du genre Romance est 118.48 minutes 
Le temps moyen du genre Film-Noir est 110.0 minutes 
Le temps moyen du genre War est 140.43 minutes 
Le temps moyen du genre Sport est 122.0 minutes 
Le temps moyen du genre W